In [ ]:
pip install pandas rdflib

In [ ]:
import pandas as pd
from rdflib import Graph

In [ ]:
# Read CSV
df_classes = pd.read_csv('FinalDataset/DBpediaClasses.csv', header=None)
df_classes.columns = ['Class', 'Subclass']

In [ ]:
df_classes = df_classes.drop(0).reset_index(drop=True)
print(df_classes.head(10))

In [ ]:
# Read the ttl with the classes relations
g_clases = Graph()
g_clases.parse('FinalDataset/instance_types_es.2017-04-01.ttl', format='ttl')

# Read ttl with texts
g_textos = Graph()
g_textos.parse('FinalDataset/long_abstracts_es.ttl.2017-04-01', format='ttl')

In [ ]:
# Create dictionary to store relations
resource_to_class = {}
resource_to_text = {}

# Extract relations of every class
for s, p, o in g_clases:
    if str(p) == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type':
        resource_to_class[str(s)] = str(o)

# Extract relations from texts
for s, p, o in g_textos:
    if str(p) == 'http://dbpedia.org/ontology/abstract':
        resource_to_text[str(s)] = str(o)

# Create a list for the new csv data
rows = []

for resource, clase in resource_to_class.items():
    clase = clase.split('/')[-1]
    if resource in resource_to_text:
        texto = resource_to_text[resource]
        # Verificar si la clase existe en el DataFrame
        subclase_series = df_classes.loc[df_classes['Class'] == clase, 'Subclass']
        if not subclase_series.empty:
            subclase = subclase_series.values[0]
            rows.append([clase, subclase, texto])

# Save the data
df_final = pd.DataFrame(rows, columns=['Class', 'Subclass', 'Text'])
df_final.to_csv('FinalDataset/final_dataset.csv', index=False)